In [1]:
import torch

# requires_grad e funzione backward

### Esempio base

In [2]:
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=True)
print(x.grad)

None


In [3]:
# eleva al quadrato tutti gli elementi della matrice
out1 = x.pow(2)
# somma tutti gli elementi della matrice
out2=out1.sum()

In [4]:
out2.backward()

In [5]:
x.grad

tensor([[ 2., -2.],
        [ 2.,  2.]])

### Se non si azzera il gradiente dopo il backward, il gradiente successivo si somma al precedente

In [6]:
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=True)
print(x.grad)

None


In [7]:
# eleva al quadrato tutti gli elementi della matrice
out1 = x.pow(2)
# somma tutti gli elementi della matrice
out2=out1.sum()

In [8]:
out2.backward()
x.grad

tensor([[ 2., -2.],
        [ 2.,  2.]])

In [9]:
# eleva al quadrato tutti gli elementi della matrice
out1 = x.pow(2)
# somma tutti gli elementi della matrice
out2=out1.sum()

In [10]:
out2.backward()
x.grad

tensor([[ 4., -4.],
        [ 4.,  4.]])

### Effetto di un assegnamento sulla backpropagation

In [11]:
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=True)
print(x.grad)

None


In [12]:
# eleva al quadrato tutti gli elementi della matrice
out1 = x.pow(2)
out1[0,0] = 0
# somma tutti gli elementi della matrice
out2=out1.sum()

In [13]:
# se x[0,0] viene incrementato dx l'incremento di out2 è comunque 0
out2.backward()
x.grad

tensor([[ 0., -2.],
        [ 2.,  2.]])

# requires_grad_, metodo per cambiare il flag require_grad di un tensore

### requires_grad_(True)

In [14]:
# per default requires_grad = False
x = torch.tensor([[1., -1.], [1., 1.]])

In [15]:
# eleva al quadrato tutti gli elementi della matrice
out1 = x.pow(2)
out1.requires_grad_()
# somma tutti gli elementi della matrice
out2=out1.sum()

In [16]:
out2.backward()

In [17]:
out1.grad

tensor([[1., 1.],
        [1., 1.]])

In [18]:
x.grad

### requires_grad_(False)

### Si può utilizzare solo su tensori foglia (cioè non prodotti da operazioni con operandi con requires_grad=True)

In [19]:
# per default requires_grad = False
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=True)

In [20]:
# eleva al quadrato tutti gli elementi della matrice
out1 = x.pow(2)
out1.requires_grad=False
# somma tutti gli elementi della matrice
out2=out1.sum()

RuntimeError: you can only change requires_grad flags of leaf variables. If you want to use a computed variable in a subgraph that doesn't require differentiation use var_no_grad = var.detach().

# Tensore foglia e metodo is_leaf

Un tensore è una foglia se **requires_grad=False** oppure se **requires_grad=True** ma il tensore è stato creato direttamente dall'utente e non è il risultato di un'operazione tra tensori in cui almeno uno degli operandi ha requires_grad = True

In [ ]:
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=True)
y = x.pow(2)

In [ ]:
x.is_leaf

In [ ]:
y.is_leaf

In [ ]:
z = y.sum()

In [ ]:
z.backward()

In [ ]:
z.grad

In [ ]:
y.grad

In [ ]:
x.grad

# Metodo retain_grad

Di default l'attributo grad viene valorizzato solo per i tensori foglia. Il metodo **retain_grad** permette di calcolare il **gradiente** anche per tensori che **non sono foglie**

In [ ]:
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=True)
y = x.pow(2)

In [ ]:
x.is_leaf

In [ ]:
y.is_leaf

In [ ]:
y.retain_grad()

In [ ]:
z = y.sum()

In [ ]:
z.retain_grad()

In [ ]:
z.backward()

In [ ]:
z.grad

In [ ]:
y.grad

In [ ]:
x.grad

# detach e detach_

### detach ritorna un nuovo tensore staccato dal grafo (attenzione! condivide gli stessi dati del tensore originale)

In [24]:
x = torch.tensor([1.,2,3], requires_grad=True)

In [26]:
y = torch.tensor([4.,5,6], requires_grad=False)

In [27]:
z = x + y

In [29]:
z.requires_grad

True

In [30]:
z1 = z.detach()

In [31]:
z1.requires_grad

False

In [32]:
z

tensor([5., 7., 9.], grad_fn=<AddBackward0>)

In [33]:
z1

tensor([5., 7., 9.])

In [34]:
z1[0] = 1

In [35]:
z

tensor([1., 7., 9.], grad_fn=<AddBackward0>)

### detach_ è come detach ma inplace

In [37]:
z.requires_grad

True

In [39]:
z.is_leaf

False

In [40]:
z.detach_()

tensor([1., 7., 9.])

In [41]:
z.requires_grad

False

In [42]:
z.is_leaf

True

# register_hook

Registers a backward hook.

The hook will be called every time a gradient with respect to the Tensor is computed. The hook should have the following signature:

hook(grad) -> Tensor or None

The hook should not modify its argument, but it can optionally return a new gradient which will be used in place of grad.

This function returns a handle with a method handle.remove() that removes the hook from the module.

In [69]:
x = torch.tensor([1.,2,3], requires_grad=True)
h_x = x.register_hook(lambda grad: grad * 2)

In [70]:
y = x*x
h_y = y.register_hook(lambda grad: print(f"y_grad = {grad}"))

In [71]:
z = y.sum()

In [72]:
z

tensor(14., grad_fn=<SumBackward0>)

In [73]:
z.backward()

y_grad = tensor([1., 1., 1.])


In [74]:
x.grad

tensor([ 4.,  8., 12.])